In [1]:
import numpy as np

In [2]:
# memoize results 
# not used here, since we run with two different inputs
# would be needed for larger n
def memoize(func):
    results = {}
    def helper(*args):
        if args not in results:
            results[args] = func(*args)
        return results[args]
    return helper

In [3]:
# read in guest relations
guest = {}
n = 0
rel = {}
with open("day13.txt", "r") as f:
    for line in f:
        s = line.split()
        s[-1] = s[-1][:-1]
        for g in (s[0], s[-1]):
            if g not in guest:
                # print g
                guest[g] = n
                n += 1
        if s[2] == 'lose':
            rel[(guest[s[0]], guest[s[-1]])] = -int(s[3])
        else:
            rel[(guest[s[0]], guest[s[-1]])] = int(s[3])
print guest

{'Mallory': 7, 'Frank': 5, 'Alice': 0, 'David': 3, 'George': 6, 'Bob': 1, 'Eric': 4, 'Carol': 2}


In [4]:
# store mutual symmetric relations in n x n array
happy = np.zeros((n, n))
for i in range(n):
    for j in range(i+1, n):
        happy[i, j] = rel[(i, j)] + rel[(j, i)]
        happy[j, i] = happy[i, j]
    
def unpack(ind):    
    return [i for i in range(n) if (1<<i) & ind]

In [5]:
# dynamic programming here:
# given a set of guests (bitcoded in ind) what is the best seating 
# if outer[0] is sitting to the far left and outer[1] to the far right
# new guests are always placed to the far right. 

#@memoize
def seating(ind, outer):
    left, right = outer
    lost = happy[left, right]
    res = []
    members = unpack(ind)
    if len(members) < 4:
        # 3 or less guests: every seating is the same
        return sum([happy[i, j] for i, j in zip(members[:], members[1:] + [members[0]]) ])
    l = ind^(1<<right)
    for i in members:
        if i == left or i == right:
            continue
        # outer guests stay the same, try every subsample where i (inner guest) was added last
        res.append( seating(l, (left, i)) - happy[left, i] + happy[i, right])
    return max(res) + happy[left, right] 

In [6]:
res = [seating((1<<n) - 1, (0, i)) for i in range(1, n)]
print max(res), res

664.0 [664.0, 520.0, 664.0, 626.0, 333.0, 514.0, 482.0]


In [11]:
dum = np.zeros((n+1, n+1))
dum[:n, :n] = happy
happy = dum
n = n + 1

res = [seating((1<<n) - 1, (0, i)) for i in range(1, n)]
print max(res), res

640.0 [640.0, 477.0, 640.0, 602.0, 395.0, 589.0, 544.0, 636.0]


In [7]:
count

8659